In [ ]:
# 下記セルを実行すると、authorization codeの入力を求められます。
# 出力されたリンク先をクリックし、Googleアカウントにログインし、
# authorization codeをコピーし、貼り付けをおこなってください。
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

import os
project = 'sample_data'
chapter = 7
os.chdir(f'/content/drive/MyDrive/{project}/chapter-{chapter}/')

# chapter 7 地理空間データ加工・可視化10ノック


## ノック81: 地理空間データの形式を理解しよう


In [ ]:
import geopandas as gpd
gdf_master = gpd.read_file('data/L01-18.shp')
gdf_master

In [ ]:
gdf_master['geometry']

In [ ]:
gdf_json = gpd.read_file('data/L01-18.geojson')
gdf_json

## ノック８２：読み込んだデータを確認しよう

In [ ]:
len(gdf_master)

In [ ]:
gdf_master.columns

In [ ]:
gdf = gdf_master[['L01_006', 'L01_023', 'geometry']].rename(columns={'L01_006': 'advertised_price', 'L01_023': 'address'})
gdf

## ノック８３：都道府県名を住所から抽出しよう

In [ ]:
import re

def extract_prefecture(address):

    match = re.search(r'([^\d]+?[都道府県])', address)
    if match:
        return match.group(1)
    else:
        return None

gdf['都道府県'] = gdf['address'].apply(extract_prefecture)

gdf['都道府県']

In [ ]:
print(gdf['都道府県'].unique())
print(gdf['都道府県'].nunique())

In [ ]:
def extract_prefecture(address):

    match = re.search('東京都|北海道|(?:京都|大阪)府|.{2,3}県', address)
    if match:
        return match.group()
    else:
        return None

gdf['都道府県'] = gdf['address'].apply(extract_prefecture)

gdf['都道府県']

In [ ]:
print(gdf['都道府県'].unique())
print(gdf['都道府県'].nunique())

## ノック８４：価格の分布を可視化してみよう

In [ ]:
!pip install japanize-matplotlib
import matplotlib.pyplot as plt
import japanize_matplotlib

gdf['advertised_price'] = gdf['advertised_price'].astype(int)
gdf_avg = gdf.groupby('都道府県')['advertised_price'].mean().reset_index()

plt.figure(figsize=(10, 9))
plt.barh(gdf_avg['都道府県'], gdf_avg['advertised_price'])
plt.title('各都道府県の平均公示価格')
plt.xlabel('平均公示価格')
plt.ylabel('都道府県')
plt.show()

## ノック８５：ポイントを表示してみよう

In [ ]:
gdf.plot(column = 'advertised_price')

In [ ]:
from matplotlib.colors import Normalize

norm = Normalize(vmin=0, vmax=1e6)

gdf_over = gdf[gdf['advertised_price'] > 500000]
gdf_over.plot(column = 'advertised_price',
                      cmap = 'Purples',
                      norm = norm,
                      legend = True)

## ノック８６：地図上にポイントを表示してみよう

In [ ]:
!pip install japanmap
from shapely.geometry import Polygon
from japanmap import get_data, pref_points

fig, ax = plt.subplots(1,1, figsize=(8, 8))
japan_poly = [Polygon(points) for points in pref_points(get_data())]
gdf_japan = gpd.GeoDataFrame(crs = 'epsg: 4612', geometry=japan_poly)

gdf_japan.plot(color = 'darkgray', ax = ax)

gdf_over.plot(column = 'advertised_price',
                      cmap = 'Purples',
                      norm = norm,
                      legend = True,
                      ax = ax,
                      s = 7)

plt.title('各都道府県の平均公示価格')
plt.show()

## ノック８７：インタラクティブな地図を作成してみよう

In [ ]:
import folium

f_map = folium.Map(location=(35.6905, 139.6995), zoom_start=15)
folium.GeoJson(gdf.to_json()).add_to(f_map)
f_map

## ノック８８：作成した地図を保存してみよう

In [ ]:
f_map.save('data/shinjuku_station_map.html')

## ノック８９：座標変換してみよう

In [ ]:
gdf.crs

In [ ]:
from osgeo import ogr, osr
import shapefile

src_srs = osr.SpatialReference()
dst_srs = osr.SpatialReference()
src_srs.ImportFromEPSG(4612)
dst_srs.ImportFromEPSG(3100)
transform = osr.CoordinateTransformation(src_srs, dst_srs)

src_advertised_price = shapefile.Reader('data/L01-18.shp')
shps_advertised_price = src_advertised_price.shapes()

utm_list = []
for shp in shps_advertised_price :
    utm_point = list(map(lambda point: transform.TransformPoint(point[1], point[0])[:2], shp.points))
    utm_list.append(utm_point)

utm_list

## ノック９０：座標間の距離を計算しよう

In [ ]:
from shapely import Point

point = Point(35.6905, 139.6995)
point = gpd.GeoDataFrame(geometry=[point], crs=2451)

gdf.to_crs(2451).distance(point.geometry[0])